In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('taxi-fare-prediction').getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 11:19:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
directory = "/home/ubuntu/working/datasource"
trip_files = "/trips/*"

trips_df = spark.read.csv(f"file:///{directory}/{trip_files}", inferSchema=True, header=True)
trips_df.printSchema()

# trip_distance: double (nullable = true) => x
# tolls_amount: double (nullable = true) => y

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [4]:
trips_df.createOrReplaceTempView("trips")

In [5]:
# 데이터 정제
query = """
SELECT
    t.trip_distance,
    t.total_amount

FROM trips t

WHERE t.total_amount < 200
  AND t.total_amount > 0
  AND t.passenger_count < 5
  AND TO_DATE(t.tpep_pickup_datetime) >= '2021-01-01'
  AND TO_DATE(t.tpep_pickup_datetime) < '2021-08-01'
  AND t.trip_distance < 10
  AND t.trip_distance > 0
"""

data_df = spark.sql(query)

In [6]:
data_df.show(5)

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         1.13|       11.16|
|         2.68|       18.59|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
+-------------+------------+
only showing top 5 rows



# train / test 분할

In [7]:
train_sdf, test_sdf = data_df.randomSplit([0.8,0.2], seed=42)

In [8]:
train_sdf.cache() #훈련데이터는 캐싱까지해주기

DataFrame[trip_distance: double, total_amount: double]

# 벡터 어셈블링

In [9]:
# 피처가 하나만 있어도 벡터 형식으로 데이터를 뭉쳐야함.

In [11]:
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(
    inputCols = ["trip_distance"],
    outputCol = "features"
)

In [13]:
vec_train_sdf = vec_assembler.transform(train_sdf)
vec_train_sdf.show(5)

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|        3.05|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 5 rows



# 모델 생성 및 훈련

In [18]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter = 50,   # 최대훈련횟수
    featuresCol = 'features',
    labelCol = 'total_amount'
)

In [20]:
lr_model = lr.fit(vec_train_sdf)

23/11/20 11:34:51 WARN Instrumentation: [5b43a8c8] regParam is zero, which might cause numerical instability and overfitting.


# 예측

In [21]:
# 테스트 데이터 변환
vec_test_sdf = vec_assembler.transform(test_sdf)
vec_test_sdf.show(5)

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 5 rows



In [22]:
# EDA는 action(show)은 많이해야함.
predictions = lr_model.transform(vec_test_sdf)
predictions.show(5)

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
+-------------+------------+--------+-----------------+
only showing top 5 rows



# 평가

In [24]:
# RMSE 확인
lr_model.summary.rootMeanSquaredError # 오차가 3달러(3천원)

3.510945426773181

In [25]:
# R^2 확인
lr_model.summary.r2 # 0.76이면 충분히 괜찮음 (70%넘어가도 잘된거임)

0.763032284678061

In [26]:
spark.stop()